<a href="https://colab.research.google.com/github/matthewLarrosa/Crop-vs-Weeds-Model/blob/main/CAP_4612.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'crop-and-weed-detection-data-with-bounding-boxes:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F589173%2F1062313%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240328%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240328T200406Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D00bbc676198ba099bd4feabe28c2b6b210ed6b4c8ab462933b3d738db1c76f9a8dfb74e0645966cc28e0ba932e9508518decb0fe8813fb85f3c6876c3c6744f1c5e9c79fda2b0173a73289e877c1fff65f21446f66583f5e0004c7b69a50eb196b7a84a38d8015258c3e85e1fe177854e8f751d6ab6d6fd2c5dea3335607d264e5bd063cb12b5382e75379404c9b7f3d528b328cbaf80c05968aa6c2f9747445b7f19bafca1823d58ac42421dedddf7b833ec8b9280c797d15229f57fc3dcdfe949074ce1fda9b879f78fc4c5f75b853694fb21c4aa0cdff433e65a2a001f7452020245646ecedc30971916eff500a002c55ab2b3d7f898c8ee5f23f3b79c8a5'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split

dataset_crop_weeds = '/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/'

img_height = 224
img_width = 224

# List to store image paths and annotations
images = []
annotations = []

for filename in os.listdir(dataset_crop_weeds):
    if filename.endswith('.jpeg'):
        image_crop_weeds = os.path.join(dataset_crop_weeds, filename)
        images.append(image_crop_weeds)
    elif filename.endswith('.txt'):
        annotation_crop_weeds = os.path.join(dataset_crop_weeds, filename)
        annotations.append(annotation_crop_weeds)

# Converts annotations to DataFrame
# data = {'Annotation': annotations}
# df = pd.DataFrame(data)

In [ ]:
# Split the data into training, validation, and test sets
train_images, test_images, train_annotations, test_annotations = train_test_split(images, annotations, test_size=0.2, random_state=42)
train_images, val_images, train_annotations, val_annotations = train_test_split(train_images, train_annotations, test_size=0.1, random_state=42)

In [ ]:
# Define the CNN model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_annotations))
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_annotations))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_annotations))


In [ ]:
# Apply preprocessing functions to dataset (define preprocess_image and preprocess_annotation functions)
train_dataset = train_dataset.map(lambda x, y: (preprocess_image(x), preprocess_annotation(y)))
val_dataset = val_dataset.map(lambda x, y: (preprocess_image(x), preprocess_annotation(y)))
test_dataset = test_dataset.map(lambda x, y: (preprocess_image(x), preprocess_annotation(y)))


NameError: in user code:

    File "/tmp/ipykernel_32/389378627.py", line 2, in None  *
        lambda x, y: (preprocess_image(x), preprocess_annotation(y))

    NameError: name 'preprocess_annotation' is not defined


In [ ]:
# Define batch size and number of epochs
batch_size = 32
epochs = 10

10

In [ ]:
# Train the model
history = model.fit(train_images, train_annotations, epochs,
                    validation_data=(val_images, val_annotations))

ValueError: Unrecognized data type: x=['/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/agri_0_6717.jpeg', '/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/agri_0_5066.jpeg', '/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/agri_0_4499.jpeg', '/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/agri_0_5128.jpeg', '/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/agri_0_1888.jpeg', '/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/agri_0_4010.jpeg', '/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/agri_0_5542.jpeg', '/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/agri_0_7878.jpeg', '/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/agri_0_4913.jpeg', '/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_annotations, verbose=2)
print("Test accuracy:", test_acc)